In [9]:
from datascience import *
import numpy as np

import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
%matplotlib inline

## The Broste Thesis

Steven Broste, and others, believed that they could prove that reducing cholesterol through dietary changes would decrease Chronic Heart Disease. 

Hypothesis: 
Replacing saturated fat (e.g. dairy) with polyunsaturated fat (e.g. plant-based oil) reduces risk of heart disease.

Justification:
* This replacement reduces serum cholesterol.
* Serum cholesterol is associated with heart disease.
* Clinical trials that used polyunsaturated fat to replace saturated fat reduced the incidence of CVD." (AHA, 2017). http://circ.ahajournals.org/content/early/2017/06/15/CIR.0000000000000510

The data recovered from the Minnesota Coronary Experiment, including Steven Broste's analysis in his master's thesis, strongly challenges the traditional diet-heart hypothesis by demonstrating that reducing cholesterol through dietary changes may not be associated with a reduction in mortality and may even be associated with increased mortality in certain populations.

Unique Opportunity: The data recovered from the Minnesota Coronary Experiment was based on changing the diet of 9423 people who institutionalized in state mental hospitals or nursing homes. This was a controlled population. 
* Control group: food was cooked in butter and served with butter, as usual.
* Treatment group: food was cooked in vegetable oil and served with margarine (corn-based substitute)

It was a double-blind study: neither the subjects nor the people delivering the treatment (food servers) knew which group the participants were in. 


In [10]:
summary = Table(['Age', 'Condition', 'Total', 'Deaths', 'CHD Deaths']).with_rows([
    ['0-34',  'Diet',    1367, 3, 0],
    ['35-44', 'Diet',    728, 3, 0],
    ['45-54', 'Diet',    767, 14, 4],
    ['55-64', 'Diet',    870, 35, 7],
    ['65+',   'Diet',    953, 190, 42],
    ['0-34',  'Control', 1337, 7, 1],
    ['35-44', 'Control', 731, 4, 1],
    ['45-54', 'Control', 816, 16, 4],
    ['55-64', 'Control', 896, 33, 12],
    ['65+',   'Control', 958, 162, 34],   
])
summary

Age,Condition,Total,Deaths,CHD Deaths
0-34,Diet,1367,3,0
35-44,Diet,728,3,0
45-54,Diet,767,14,4
55-64,Diet,870,35,7
65+,Diet,953,190,42
0-34,Control,1337,7,1
35-44,Control,731,4,1
45-54,Control,816,16,4
55-64,Control,896,33,12
65+,Control,958,162,34


In [11]:
sum(summary.column('Total'))

#Problem: We can't do a permutation test with a summary. We need one row for each individual. 

9423

In [12]:
#We need to assign a death value based on the known proportion of deaths in a group. 
#We can use arange to create an array of true, false values. 

np.arange(12) < 3

array([ True,  True,  True, False, False, False, False, False, False,
       False, False, False], dtype=bool)

In [13]:
#We use that arange command to create an array of deaths (True) based on the number who died in that age group. 
#Can you find the use of the array and how it assigns the correct value for True's in the code belows?

subjects = Table(['Age', 'Condition', 'Participated', 'Died'])
for row in summary.rows:
    i = np.arange(0, row.item('Total'))
    t = Table().with_columns('Died', i < row.item('Deaths'))
    t.append_column('Age', row.item('Age'))
    t.append_column('Condition', row.item('Condition'))
    t.append_column('Participated', True) #Since everyone participated. 
    subjects.append(t)
subjects

Age,Condition,Participated,Died
0-34,Diet,True,True
0-34,Diet,True,True
0-34,Diet,True,True
0-34,Diet,True,False
0-34,Diet,True,False
0-34,Diet,True,False
0-34,Diet,True,False
0-34,Diet,True,False
0-34,Diet,True,False
0-34,Diet,True,False


In [14]:
#Does the summary of the newly created table match the original information. 

subjects.group(['Age', 'Condition'], sum)

Age,Condition,Participated sum,Died sum
0-34,Control,1337,7
0-34,Diet,1367,3
35-44,Control,731,4
35-44,Diet,728,3
45-54,Control,816,16
45-54,Diet,767,14
55-64,Control,896,33
55-64,Diet,870,35
65+,Control,958,162
65+,Diet,953,190


Now we can conduct an A/B Test.

First check the difference between the death rates of people on the dieat and control group. </br>
Next radomize the Condition assignment then check that difference again. 

In [15]:
#Choose your statistic.
#We want the difference between those who died in each group. 

def hazard_rate(counts):
    return counts.item('Died sum') / counts.item('Participated sum')

def rate_difference(t):
    counts = t.drop('Age').group('Condition', sum)
    return abs(hazard_rate(counts.row(1)) - hazard_rate(counts.row(0)))

rate_difference(subjects)

0.0054393439270044933

In [16]:
#Check difference for the youngest subjects.

rate_difference(subjects.where('Age', '0-34'))

0.0030410154080667343

In [17]:
#Check difference for the oldest subjects. 

rate_difference(subjects.where('Age', '65+'))

0.030268112783058437

### Run a Hypothesis Test

Compute the observed absolute difference in hazard rates, for each age group, between the Control and Diet groups. 

Then compute the p-value to determine if that difference is by chance or not. 


In [18]:
def test(t):
    observed = rate_difference(t)
    repetitions = 200

    stats = make_array()
    for i in np.arange(repetitions):
        simulated_results = t.select('Died').sample().column('Died')
        simulated_outcomes = t.with_column('Died', simulated_results)
        simulated_stat = rate_difference(simulated_outcomes)
        stats = np.append(stats, simulated_stat)

    # Find the empirical P-value:
    p = np.count_nonzero(stats >= observed) / repetitions
    
    print('Observed absolute difference in hazard rates:', observed)
    print('P-value:', p)

#test(subjects)

In [19]:
for age in subjects.group('Age').column('Age'):
    print('Ages', age)
    test(subjects.where('Age', age))

Ages 0-34
Observed absolute difference in hazard rates: 0.00304101540807
P-value: 0.19
Ages 35-44
Observed absolute difference in hazard rates: 0.00135107710347
P-value: 0.78
Ages 45-54
Observed absolute difference in hazard rates: 0.00135490963008
P-value: 0.83
Ages 55-64
Observed absolute difference in hazard rates: 0.00339952791461
P-value: 0.765
Ages 65+
Observed absolute difference in hazard rates: 0.0302681127831
P-value: 0.145


Only the oldest group has a p-value that may make us want to do the experiment again to see if we could get a smaller p-value. All the other groups have values clearly in excess of any p-value cutoff. 

### Conclusion
Replacement of saturated fat in the diet with linoleic acid effectively lowers serum cholesterol but does not support the hypothesis that this translates to a lower risk of death from coronary heart disease or all causes. Findings from the Minnesota Coronary Experiment add to growing evidence that incomplete publication has contributed to overestimation of the benefits of replacing saturated fat with vegetable oils rich in linoleic acid.